In [1]:
import sys
from sys import exit
from pathlib import Path
from os import getcwd
from os.path import join

sys.path.append(str(Path(getcwd()).parent))

In [2]:
import codecs
import pandas as pd
import numpy as np
from bulletin.services.metabase import Metabase
from bulletin import default_output

mb = Metabase()

pd.set_option('display.max_columns', None)


---------------------------------------------------------------

root: /home/vinnylg/projects/covid19datascience/bulletin
parent: /home/vinnylg/projects/covid19datascience
default_input: /home/vinnylg/projects/covid19datascience/input
default_output: /home/vinnylg/projects/covid19datascience/output

---------------------------------------------------------------

Cookie:'8d43f********-****-****-****-************5aece'

saved_queries: ['diario', 'old_diario', 'all_notifica'] 



In [3]:
notificacao, information_schema, constraint = mb.download_tables()

Success code 200
Download finish, time elapsed: 0:00:00.454641
downloaded shape (179, 3)

Success code 200
Download finish, time elapsed: 0:00:00.555140
downloaded shape (4112, 3)

Success code 200
Download finish, time elapsed: 0:00:01.719933
downloaded shape (219, 5)

Download tables time elepsed:00:00:03


In [4]:
notificacao.groupby(['table_name','udt_name']).count()

column_name  constraint_name  foreign_table_name  \
table_name  udt_name                                                     
notificacao datetime           21               21                  21   
            int                91               91                  91   
            str                67               67                  67   

                      foreign_column_name  
table_name  udt_name                       
notificacao datetime                   21  
            int                        91  
            str                        67

In [7]:
with codecs.open('tables.dbml',"w+","utf-8-sig") as out:
    out.seek(0)
    for table_name, table in information_schema.groupby('table_name'):
        if table_name in notificacao['foreign_table_name'].unique():
            out.write(f"Table {table_name}" + " {\n")
            for column_name, udt_name in zip(table['column_name'],table['udt_name']):
                if column_name in constraint.loc[constraint['foreign_table_name']==table_name,'foreign_column_name'].values: 
                    out.write(f"\t {column_name} {udt_name} [pk]\n")
                    notificacao.loc[notificacao['foreign_table_name']==table_name,'ref'] = f"[ref: - {table_name}.{column_name}]"
                else:
                    out.write(f"\t {column_name} {udt_name}\n")
            out.write("}\n\n")
    
    notificacao.loc[notificacao['ref'].isna(),'ref'] = ''

    out.write(f"Table notificacao" + " {\n")
    for column_name, udt_name,ref in zip(notificacao['column_name'],notificacao['udt_name'],notificacao['ref']):
        out.write(f"\t {column_name} {udt_name} {ref}\n")
    out.write("}\n\n")

In [8]:
notificacao.columns = ['table','column','dtypes','merge_name','merge_table','merge_column','ref']

In [9]:
notificacao['use_cols'] = ''
notificacao['default_values'] = '' 
notificacao['converters'] = '' 
notificacao = notificacao[['table','column','use_cols','dtypes','converters','default_values','merge_name','merge_table','merge_column','ref']]
notificacao.to_excel(join(default_output,'colunas_notifica.xlsx'))
notificacao

,table,column,use_cols,dtypes,converters,default_values,merge_name,merge_table,merge_column,ref
0,notificacao,id,,int,,,,,,
1,notificacao,data_notificacao,,datetime,,,,,,
2,notificacao,possui_cpf,,int,,,possui_cpf,sim_nao,id,[ref: - sim_nao.id]
3,notificacao,tipo_paciente,,int,,,tipo_paciente,tipo_paciente,id,[ref: - tipo_paciente.id]
4,notificacao,pais_residencia,,str,,,pais_residencia,pais,co_pais,[ref: - pais.co_pais]
...,...,...,...,...,...,...,...,...,...,...
174,notificacao,lote_covid19_1a,,str,,,,,,
175,notificacao,data_covid19_2a,,datetime,,,,,,
176,notificacao,lote_covid19_2a,,str,,,,,,
177,notificacao,vacina_influenza,,int,,,,,,
